CIFAR-10 is loaded using the keras datasets interface. The per-pixel mean is subtracted as preprocessing. The code in this notebook is based on the keras example found here: https://github.com/keras-team/keras/blob/master/examples/cifar10_resnet.py

In [1]:
import os
import numpy as np
import keras
from keras.datasets import cifar10

# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_test -= x_train_mean

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

Using TensorFlow backend.


The model is created the same way as cifar10_resnet.py does. Checkpoints are saved in the "saved_models" directory. The model itself is a 29 layer deep version of resnet v2. More details can be found in cifar10_resnet.py.

In [3]:
from cifar10_resnet import *


n = 3
depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, 2)

model = resnet_v2(input_shape=input_shape, depth=depth)
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=lr_schedule(0)),
    metrics=['accuracy'])

# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(
    filepath=filepath,
    monitor='val_acc',
    verbose=1,
    save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(
    factor=np.sqrt(0.1),
    cooldown=0,
    patience=5,
    min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

W0228 18:28:45.504156 140291341621056 deprecation.py:506] From /home/ryan-desktop/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/training/moving_averages.py:210: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Learning rate:  0.001


Data augmentation is used to improve performance. The model took approximately 3-3.5 hours to train and reached an accuracy of 92.25%

In [19]:
datagen = ImageDataGenerator(
    # set input mean to 0 over the dataset
    featurewise_center=False,
    # set each sample mean to 0
    samplewise_center=False,
    # divide inputs by std of dataset
    featurewise_std_normalization=False,
    # divide each input by its std
    samplewise_std_normalization=False,
    # apply ZCA whitening
    zca_whitening=False,
    # epsilon for ZCA whitening
    zca_epsilon=1e-06,
    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=0,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    # set range for random shear
    shear_range=0.,
    # set range for random zoom
    zoom_range=0.,
    # set range for random channel shifts
    channel_shift_range=0.,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # value used for fill_mode = "constant"
    cval=0.,
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False,
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(
    datagen.flow(x_train, y_train, batch_size=32),
    steps_per_epoch=60000 // 32,
    validation_data=(x_test, y_test),
    epochs=200, 
    verbose=1, 
    workers=4,
    callbacks=callbacks)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/200
Learning rate:  0.001
1875/1875 [==============================] - 62s 33ms/step - loss: 1.6784 - acc: 0.5432 - val_loss: 1.4523 - val_acc: 0.6000

Epoch 00001: val_acc improved from -inf to 0.60000, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/cifar10_ResNet29v2_model.001.h5
Epoch 2/200
Learning rate:  0.001
1875/1875 [==============================] - 62s 33ms/step - loss: 1.2609 - acc: 0.6642 - val_loss: 1.5009 - val_acc: 0.5789

Epoch 00002: val_acc did not improve from 0.60000
Epoch 3/200
Learning rate:  0.001
1875/1875 [==============================] - 61s 33ms/step - loss: 1.0952 - acc: 0.7161 - val_loss: 1.2585 - val_acc: 0.6657

Epoch 00003: val_acc improved from 0.60000 to 0.66570, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/cifar10_ResNet29v2_model.003.h5
Epoch 4/200
Learning rate:  0.001
1875/1875 [==============================] - 62s 33ms/step - loss: 1.0111 - acc: 0.7430 - val_loss: 1.2

1875/1875 [==============================] - 60s 32ms/step - loss: 0.5032 - acc: 0.8984 - val_loss: 0.7287 - val_acc: 0.8375

Epoch 00068: val_acc did not improve from 0.86570
Epoch 69/200
Learning rate:  0.001
1875/1875 [==============================] - 61s 32ms/step - loss: 0.5063 - acc: 0.8970 - val_loss: 0.6640 - val_acc: 0.8508

Epoch 00069: val_acc did not improve from 0.86570
Epoch 70/200
Learning rate:  0.001
1875/1875 [==============================] - 61s 33ms/step - loss: 0.4950 - acc: 0.8994 - val_loss: 0.6297 - val_acc: 0.8613

Epoch 00070: val_acc did not improve from 0.86570
Epoch 71/200
Learning rate:  0.001
1875/1875 [==============================] - 62s 33ms/step - loss: 0.5003 - acc: 0.8984 - val_loss: 0.6687 - val_acc: 0.8520

Epoch 00071: val_acc did not improve from 0.86570
Epoch 72/200
Learning rate:  0.001
1875/1875 [==============================] - 61s 33ms/step - loss: 0.4926 - acc: 0.9009 - val_loss: 0.7570 - val_acc: 0.8380

Epoch 00072: val_acc did not i

1875/1875 [==============================] - 60s 32ms/step - loss: 0.1760 - acc: 0.9833 - val_loss: 0.4130 - val_acc: 0.9203

Epoch 00138: val_acc did not improve from 0.92100
Epoch 139/200
Learning rate:  1e-05
1875/1875 [==============================] - 60s 32ms/step - loss: 0.1759 - acc: 0.9837 - val_loss: 0.4152 - val_acc: 0.9206

Epoch 00139: val_acc did not improve from 0.92100
Epoch 140/200
Learning rate:  1e-05
1875/1875 [==============================] - 60s 32ms/step - loss: 0.1769 - acc: 0.9829 - val_loss: 0.4144 - val_acc: 0.9212

Epoch 00140: val_acc improved from 0.92100 to 0.92120, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/cifar10_ResNet29v2_model.140.h5
Epoch 141/200
Learning rate:  1e-05
1875/1875 [==============================] - 60s 32ms/step - loss: 0.1745 - acc: 0.9840 - val_loss: 0.4130 - val_acc: 0.9207

Epoch 00141: val_acc did not improve from 0.92120
Epoch 142/200
Learning rate:  1e-05
1875/1875 [=======================

The modified version of resnet, replacing the classifier with the fuzzy classifier. It is trained with the same parameters as the normal version of resnet. It was able to achieve 92.56% accuracy, slightly better than the traditional version, but this may be due to random chance.

In [7]:
from model import LogGaussMF


def resnet_hybrid(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    y = LogGaussMF(num_classes)(y)
    y = keras.layers.Lambda(lambda x: K.sum(x, axis=-1))(y)
    outputs = keras.layers.Activation("softmax")(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


n = 3
depth = n * 9 + 2

# Model name, depth and version
model_type = 'fuzzy-ResNet%dv%d' % (depth, 2)

model = resnet_hybrid(input_shape=input_shape, depth=depth)
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=lr_schedule(0)),
    metrics=['accuracy'])

# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(
    filepath=filepath,
    monitor='val_acc',
    verbose=1,
    save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(
    factor=np.sqrt(0.1),
    cooldown=0,
    patience=5,
    min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

Learning rate:  0.001


In [8]:
datagen = ImageDataGenerator(
    # set input mean to 0 over the dataset
    featurewise_center=False,
    # set each sample mean to 0
    samplewise_center=False,
    # divide inputs by std of dataset
    featurewise_std_normalization=False,
    # divide each input by its std
    samplewise_std_normalization=False,
    # apply ZCA whitening
    zca_whitening=False,
    # epsilon for ZCA whitening
    zca_epsilon=1e-06,
    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=0,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    # set range for random shear
    shear_range=0.,
    # set range for random zoom
    zoom_range=0.,
    # set range for random channel shifts
    channel_shift_range=0.,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # value used for fill_mode = "constant"
    cval=0.,
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False,
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(
    datagen.flow(x_train, y_train, batch_size=32),
    steps_per_epoch=60000 // 32,
    validation_data=(x_test, y_test),
    epochs=200, 
    verbose=1, 
    workers=4,
    callbacks=callbacks)

Epoch 1/200
Learning rate:  0.001
1875/1875 [==============================] - 74s 40ms/step - loss: 1.7039 - acc: 0.4654 - val_loss: 1.3858 - val_acc: 0.5844

Epoch 00001: val_acc improved from -inf to 0.58440, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/cifar10_fuzzy-ResNet29v2_model.001.h5
Epoch 2/200
Learning rate:  0.001
1875/1875 [==============================] - 64s 34ms/step - loss: 1.2717 - acc: 0.6208 - val_loss: 1.1939 - val_acc: 0.6516

Epoch 00002: val_acc improved from 0.58440 to 0.65160, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/cifar10_fuzzy-ResNet29v2_model.002.h5
Epoch 3/200
Learning rate:  0.001
1875/1875 [==============================] - 65s 34ms/step - loss: 1.1155 - acc: 0.6804 - val_loss: 1.1672 - val_acc: 0.6718

Epoch 00003: val_acc improved from 0.65160 to 0.67180, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/cifar10_fuzzy-ResNet29v2_model.003.h5
E

1875/1875 [==============================] - 63s 33ms/step - loss: 0.5629 - acc: 0.8749 - val_loss: 0.9478 - val_acc: 0.7776

Epoch 00033: val_acc did not improve from 0.82740
Epoch 34/200
Learning rate:  0.001
1875/1875 [==============================] - 63s 33ms/step - loss: 0.5561 - acc: 0.8776 - val_loss: 0.8977 - val_acc: 0.7903

Epoch 00034: val_acc did not improve from 0.82740
Epoch 35/200
Learning rate:  0.001
1875/1875 [==============================] - 63s 33ms/step - loss: 0.5518 - acc: 0.8781 - val_loss: 0.7180 - val_acc: 0.8371

Epoch 00035: val_acc improved from 0.82740 to 0.83710, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/cifar10_fuzzy-ResNet29v2_model.035.h5
Epoch 36/200
Learning rate:  0.001
1875/1875 [==============================] - 63s 33ms/step - loss: 0.5464 - acc: 0.8805 - val_loss: 0.9437 - val_acc: 0.7795

Epoch 00036: val_acc did not improve from 0.83710
Epoch 37/200
Learning rate:  0.001
1875/1875 [=====================

Epoch 103/200
Learning rate:  0.0001
1875/1875 [==============================] - 63s 33ms/step - loss: 0.2310 - acc: 0.9684 - val_loss: 0.4040 - val_acc: 0.9172

Epoch 00103: val_acc did not improve from 0.92080
Epoch 104/200
Learning rate:  0.0001
1875/1875 [==============================] - 63s 33ms/step - loss: 0.2300 - acc: 0.9674 - val_loss: 0.3995 - val_acc: 0.9211

Epoch 00104: val_acc improved from 0.92080 to 0.92110, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/cifar10_fuzzy-ResNet29v2_model.104.h5
Epoch 105/200
Learning rate:  0.0001
1875/1875 [==============================] - 63s 34ms/step - loss: 0.2271 - acc: 0.9692 - val_loss: 0.4093 - val_acc: 0.9180

Epoch 00105: val_acc did not improve from 0.92110
Epoch 106/200
Learning rate:  0.0001
1875/1875 [==============================] - 63s 34ms/step - loss: 0.2237 - acc: 0.9691 - val_loss: 0.3954 - val_acc: 0.9222

Epoch 00106: val_acc improved from 0.92110 to 0.92220, saving model to /ho

1875/1875 [==============================] - 63s 33ms/step - loss: 0.1694 - acc: 0.9824 - val_loss: 0.3935 - val_acc: 0.9245

Epoch 00137: val_acc improved from 0.92430 to 0.92450, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/cifar10_fuzzy-ResNet29v2_model.137.h5
Epoch 138/200
Learning rate:  1e-05
1875/1875 [==============================] - 63s 34ms/step - loss: 0.1689 - acc: 0.9823 - val_loss: 0.3951 - val_acc: 0.9235

Epoch 00138: val_acc did not improve from 0.92450
Epoch 139/200
Learning rate:  1e-05
1875/1875 [==============================] - 63s 33ms/step - loss: 0.1694 - acc: 0.9827 - val_loss: 0.3902 - val_acc: 0.9256

Epoch 00139: val_acc improved from 0.92450 to 0.92560, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/cifar10_fuzzy-ResNet29v2_model.139.h5
Epoch 140/200
Learning rate:  1e-05
1875/1875 [==============================] - 63s 34ms/step - loss: 0.1682 - acc: 0.9832 - val_loss: 0.3935 - val_acc


Epoch 00174: val_acc did not improve from 0.92560
Epoch 175/200
Learning rate:  1e-06
1875/1875 [==============================] - 63s 33ms/step - loss: 0.1599 - acc: 0.9847 - val_loss: 0.3940 - val_acc: 0.9252

Epoch 00175: val_acc did not improve from 0.92560
Epoch 176/200
Learning rate:  1e-06
1875/1875 [==============================] - 63s 34ms/step - loss: 0.1607 - acc: 0.9844 - val_loss: 0.3930 - val_acc: 0.9255

Epoch 00176: val_acc did not improve from 0.92560
Epoch 177/200
Learning rate:  1e-06
1875/1875 [==============================] - 63s 34ms/step - loss: 0.1611 - acc: 0.9846 - val_loss: 0.3932 - val_acc: 0.9245

Epoch 00177: val_acc did not improve from 0.92560
Epoch 178/200
Learning rate:  1e-06
1875/1875 [==============================] - 63s 33ms/step - loss: 0.1593 - acc: 0.9854 - val_loss: 0.3929 - val_acc: 0.9249

Epoch 00178: val_acc did not improve from 0.92560
Epoch 179/200
Learning rate:  1e-06
1875/1875 [==============================] - 63s 33ms/step - loss